In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup as bs

In [3]:
import time

In [4]:
import pandas as pd

<h1>hh.ru</h1>


In [5]:
vacancy = "Python junior"


areas = {'Moscow': 1, 
         'Saint-Petersburg': 2,
        }

In [6]:
url_hh = 'https://hh.ru/'
url_hh_search = 'https://hh.ru/search/vacancy'

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
}

params = {
    'sso_failed': '',
}

search_params_first_page = {
    'sso_failed': '',
    'text': vacancy,
    'area': areas.values(),
    'from': 'suggest_post',
    'search_field': ['name', 'company_name', 'description'],
    'L_save_area': 'true',
    'items_on_page': 20, 
    # items_on_page - важный параметр, если его не задать, 
    # то возвращаться может больше 20 значений, но bs почему-то видит только 20
}

In [7]:
def get_search_result_count(dom) -> int:
    search_header = dom.find_all('div', {'data-qa':"vacancies-search-header"})
    count = (search_header[0].find('h1').text).split()[0]
    return int(count)

In [8]:
def get_next_page_url(dom) -> str:
    next_page_url = dom.find('a', {'data-qa': 'pager-next'})
    if next_page_url:
        next_page_url = url_hh + next_page_url['href']
    else:
        next_page_url = None
    
    return next_page_url

In [9]:
def parse_vacansies(dom) -> list:
    result = []
    vacancy_serp_items = dom.find_all('div', {'class':"vacancy-serp-item__layout"})
    
    for vacancy in vacancy_serp_items:
        main_info = vacancy.find('div', {'class':'vacancy-serp-item-body__main-info'})
        
        vacancy_name = main_info.find('a', {'class':'serp-item__title'}).text
        vacancy_salary = main_info.find('span', {'data-qa':'vacancy-serp__vacancy-compensation'})
        vacancy_url = main_info.find('a', {'class':'serp-item__title'})['href']
        
        employer_name = main_info.find('a', {'data-qa':'vacancy-serp__vacancy-employer'}).text
        employer_info = url_hh + main_info.find('a', {'data-qa':'vacancy-serp__vacancy-employer'})['href']
        
        vacancy_location = main_info.find('div', {'data-qa':'vacancy-serp__vacancy-address'}).text
        
        vacancy_description = vacancy.find('div', {'class':'g-user-content'}).text
        
        vacancy_salary_currency = None
        vacancy_salary_min_value = None
        vacancy_salary_max_value = None
        
        
        if vacancy_salary:
            salary = vacancy_salary.text.split()
            if salary[0] == 'от':
                vacancy_salary_min_value = ''
                
                for i in salary[1:-1]:
                    vacancy_salary_min_value += i
                
                vacancy_salary_min_value = int(vacancy_salary_min_value)
                
            elif salary[0] == 'до':
                vacancy_salary_max_value = ''
                
                for i in salary[1:-1]:
                    vacancy_salary_max_value += i   
                
                vacancy_salary_max_value = int(vacancy_salary_max_value)
                
            else:
                for i in range(len(salary)-1):
                    if salary[i] == '–':
                        vacancy_salary_min_value = ''  
                        vacancy_salary_max_value = ''
                        
                        for j in range(0,i):
                            vacancy_salary_min_value += salary[j]
                            
                        for j in range(i+1,len(salary)-1):
                            vacancy_salary_max_value += salary[j]
                
                        vacancy_salary_min_value = int(vacancy_salary_min_value)
                        vacancy_salary_max_value = int(vacancy_salary_max_value)
            
            vacancy_salary_currency = salary[-1]
            
        result.append({
            'name': vacancy_name,
            'salary_min': vacancy_salary_min_value,
            'salary_max': vacancy_salary_max_value,
            'salary_currency': vacancy_salary_currency,
            'employer_name': employer_name,
            'employer_info': employer_info,
            'location': vacancy_location,
            'url': vacancy_url,
            'description': vacancy_description
        })
            
    return result


In [10]:
def get_vacansies_from_hh(url, headers, params, page_number=0) -> list:
    #возможно можно меньше задать
    time.sleep(2)
    
    response = requests.get(url, headers=headers, params=params)
    print(f'check page {page_number}:\n {response.url}')
    if response.status_code == 200:
        soup = bs(response.content, 'html.parser')
               
        vacansies = parse_vacansies(soup)
        
        next_page_url = get_next_page_url(soup)
        
        if next_page_url:
            vacansies += get_vacansies_from_hh(next_page_url, headers, None, page_number+1)
            return vacansies
        else:
            return vacansies
    else:
        print(response.status_code)


In [11]:
vacancies = get_vacansies_from_hh(url_hh_search, headers, search_params_first_page)

print('\n'+ '-'*25 + ' Результат ' + '-'*25)
print(f'\nПо данному запросу найдено {len(vacancies)} вакансий \n')
print('-'*61 + '\n')

for i, vacancy in enumerate(vacancies):
    print(f'Вакансия {i+1}')
    print('-'*60)
    
    for k, v in vacancy.items():
        print(f'{k}: {v}')
    print('\n')

check page 0:
 https://hh.ru/search/vacancy?sso_failed=&text=Python+junior&area=1&area=2&from=suggest_post&search_field=name&search_field=company_name&search_field=description&L_save_area=true&items_on_page=20
check page 1:
 https://hh.ru/search/vacancy?sso_failed=&text=Python+junior&area=1&area=2&from=suggest_post&search_field=name&search_field=company_name&search_field=description&L_save_area=true&items_on_page=20&page=1&hhtmFrom=vacancy_search_list
check page 2:
 https://hh.ru/search/vacancy?sso_failed=&text=Python+junior&area=1&area=2&from=suggest_post&search_field=name&search_field=company_name&search_field=description&L_save_area=true&items_on_page=20&page=2&hhtmFrom=vacancy_search_list
check page 3:
 https://hh.ru/search/vacancy?sso_failed=&text=Python+junior&area=1&area=2&from=suggest_post&search_field=name&search_field=company_name&search_field=description&L_save_area=true&items_on_page=20&page=3&hhtmFrom=vacancy_search_list
check page 4:
 https://hh.ru/search/vacancy?sso_fail


Вакансия 52
------------------------------------------------------------
name: Junior-Middle Разработчик Python на платежную платформу
salary_min: 140000
salary_max: None
salary_currency: руб.
employer_name: ООО Процессинговая Компания Пэйтерра
employer_info: https://hh.ru//employer/4137942?hhtmFrom=vacancy_search_list
location: Москва
url: https://hh.ru/vacancy/73893491?from=vacancy_search_list&query=Python+junior
description: анализировать процессы работы службы поддержки и формировать предложения по автоматизации. - разрабатывать автоматизированные решения (писать скрипты, разработка бэкофис админки), упрощающие внутренние...Опыт владения Python от полугода. Знание Django будет плюсом. Понимание языка SQL (Postgres). Опыт работы от полугода.


Вакансия 53
------------------------------------------------------------
name: Junior + / Middle Data Analyst (AntiFraud)
salary_min: None
salary_max: None
salary_currency: None
employer_name: Райффайзен Банк
employer_info: https://hh.ru//emp

In [14]:
df = pd.DataFrame(vacancies)
df.tail()

,name,salary_min,salary_max,salary_currency,employer_name,employer_info,location,url,description
167,Тимлид в команду разработки на Java,NaN,NaN,None,Яндекс,https://hh.ru//employer/1740?hhtmFrom=vacancy_...,Санкт-Петербург,https://hh.ru/vacancy/73443834?from=vacancy_se...,Управляли командой разработки. Нанимали людей ...
168,Тимлид в команду разработки на Java,NaN,NaN,None,Яндекс,https://hh.ru//employer/1740?hhtmFrom=vacancy_...,Москва,https://hh.ru/vacancy/73443817?from=vacancy_se...,Управляли командой разработки. Нанимали людей ...
169,Senior QA Engineer,NaN,3500.0,USD,Food One,https://hh.ru//employer/5952598?hhtmFrom=vacan...,Москва,https://hh.ru/vacancy/73655442?from=vacancy_se...,Построение процесса тестирования с нуля. Тести...
170,Системный администратор Linux / Junior DevOps,NaN,NaN,None,ООО Лаборатория Цифрового Зрения,https://hh.ru//employer/2412748?hhtmFrom=vacan...,"Санкт-Петербург, Площадь Ленина",https://hh.ru/vacancy/73532299?from=vacancy_se...,Выявление и анализ работы ошибок ИС. Автоматиз...
171,Ментор по Java (middle/senior),25000.0,NaN,руб.,Your CodeReview,https://hh.ru//employer/5962259?hhtmFrom=vacan...,Москва,https://hh.ru/vacancy/73439470?from=vacancy_se...,Проводить собеседования и консультации разных ...


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             172 non-null    object 
 1   salary_min       26 non-null     float64
 2   salary_max       28 non-null     float64
 3   salary_currency  36 non-null     object 
 4   employer_name    172 non-null    object 
 5   employer_info    172 non-null    object 
 6   location         172 non-null    object 
 7   url              172 non-null    object 
 8   description      172 non-null    object 
dtypes: float64(2), object(7)
memory usage: 12.2+ KB
